In [1]:
'''Loading libraries'''
import pandas as pd 
import numpy as np 
from scipy.stats import gamma, dirichlet, beta, nbinom

'''
Notes: 
1) Gamma distribution: it's not clear in the paper if they are talking about scale or rate
'''


'''Important parameters I need to constantly change'''
k = 100


In [2]:
'''Loading dataset'''
#filename = "C:\\Users\\raoki\\Documents\\GitHub\\project_spring2019\\Data\\data_final.csv"
#filename = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\data_final.csv"
filename = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\data_final_sub.csv"
data = pd.read_csv(filename, sep=',')


In [3]:
'''Saving time in the first part'''
data = data.iloc[0:30, 0:30]
data.head()

,Unnamed: 0,patients,gender,y,abr_ACC,abr_BLCA,abr_CHOL,abr_ESCA,abr_HNSC,abr_LGG,...,g100133144,g100134869,g10357,g10431,g136542,g155060,g26823,g280660,g317712,g340602
0,1,TCGA-21-A5DI,0,0,0,0,0,0,0,0,...,26,27,155,1379,0,584,0,0,0,0
1,2,TCGA-22-A5C4,0,0,0,0,0,0,0,0,...,102,44,270,2246,0,667,25,0,0,3
2,3,TCGA-2A-A8VL,0,0,0,0,0,0,0,0,...,40,34,157,2546,0,612,0,0,0,1
3,4,TCGA-2A-A8VO,0,0,0,0,0,0,0,0,...,29,21,93,1300,0,569,0,0,0,1
4,5,TCGA-2A-A8VT,0,0,0,0,0,0,0,0,...,133,161,257,1585,0,1985,3,0,0,1


In [4]:
'''Organizing columns names'''
lr = data.columns[[2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]]
y = data.columns[3]
remove = data.columns[[0,1]]
data_complete = data.copy()

In [5]:
'''
Class to work with model parameters
I thought about using the default values as chain starting values, 
however, i encouter problems to change the size of arrays and matrices 
according with my currently k
'''
class parameters:
    def __init__(self, latent_v,latent_sk,latent_pj,latent_phi ,latent_tht, prediction):
        self.ln = latent_v #array with parameters that are only one number [0-c0,1-gamma0,2-eta,3-cj] 
        self.la_sk = latent_sk #array 
        self.la_pj = latent_pj #array
        self.lm_phi = latent_phi #matrix (jk)
        self.lm_tht = latent_tht #matrix  (kv)      
        self.p = prediction #array [intercept, gender, 15 cancer types, k genes]


In [6]:
'''
Declaring my priori values: param.ln = [0-c0,1-gamma0,2-eta,3-cj], param.la_sk, param.la_pj, 
param.lm_phi, param.lm_tht and param.p = [] 
#j=sample of patients 
#v=genes
#k=latent
#UPDATE PARAM 
'''
def priori_l(param):
    k = len(param.la_sk)
    j = len(param.la_pj)
    v = param.lm_phi.shape[0]
    #Prioris on the K plate
    s0 = 1 #define better numbers in the future
    t0 = 1 #define better numbers in the future 
    c0 = gamma.pdf(x=param.ln[0],a = s0,scale = t0) 
    a0 = 1 #define better numbers in the futureT
    b0 = 1 #define better numbers in the future
    gamma0 = gamma.pdf(x=param.ln[1],a = a0,scale = b0)
    sk = gamma.pdf(x=param.la_sk,a=param.ln[1]/k,scale=param.ln[0])#array(k) gamma(3,5): mean is 15
    s0 = 1
    w0 = 1
    eta = gamma.pdf(x=param.ln[2], a = s0, scale = w0) #1
    #print('param.lm_phi',param.lm_phi.shape,'np.repeat(eta,j)',np.repeat(eta,v).shape)
    #print('line should sun to 1')
    #print('lm_phi first line',param.lm_phi[0])
    print('eta',eta)
    print(param.lm_phi)
    phivk = dirichlet.pdf(x=param.lm_phi, alpha=np.repeat(param.ln[2],v)) #MATRIX #1
    priorik = np.log(c0)+np.log(gamma0)+np.log(sk).sum()+np.log(eta)+np.log(phivk).sum()
    #priories on the J plate
    a0 = 1 #define better numbers in the future
    b0 = 1 #define better numbers in the future
    pj = beta(x=param.la_pj,a=a0,b=b0) #same numbers? #array
    e0 = 2
    f0 = 9
    cj = gamma.pdf(param.ln[3], a = e0, scale = f0)
    priorij = np.log(pj).sum()+np.log(cj) #array    
    #prioris on K and J plate
    thetakj = gamma.pdf(x=param.lm_tht,a=sk,scale = cj) #MATRIX
    priorikj = np.log(thetakj).sum()
    return (priorik+priorij+priorikj)
    
def priori_p(param):
    '''
    #add prioris of logistic regression 
    #features are the theta_kj parameters and clinical info
    #one parameter for each latent variable + one parameter for each clinical features
    '''
    k = len(param.la_sk)
    mean = len(param.p)
    sd = 5
    mean.expend(1,np.repeat(1/15, 15))
    sd.expend(0.05,np.repeat(0.05/15,15))
    mean.expend(np.repeat(1,k))
    sd.expend(np.repeat(0.05,k))
    lr = normal.pdf(param.p,loc = mean, scale = sd) #loc/mean, scale/sd

In [7]:
'''
Declare all the likelihood distributions in two separete functions
#_F represents the factor model likelihood and _P the prediction model likelihood
'''

#PASS DATA ONLY WITH RELEVANT COLUMNS FOR THIS PROBLEM 
def likelihood_F(param,dataf):
    #print(dataf.head())
    '''phi(jk) x theta(kv) must be matrix , final dim is j patients x v genes '''
    #phi_theta = np.matmul(param.lm_phi, param.lm_tht) 
    #phi_theta = np.transpose(np.matmul(param.lm_phi, param.lm_tht)) 
    phi_theta = np.dot(param.lm_phi, param.lm_tht)
    phi_theta = np.transpose(phi_theta)
    #data_F = data.drop(lr,axis = 1)
    #data_F = data_F.drop(y,axis = 1)
    #print('lf: lm_phi ', start.lm_phi.shape, ' lm_tht shape',start.lm_tht.shape )
    la_pj = np.repeat(start.la_pj, dataf.shape[1]).reshape(start.la_pj.shape[0],dataf.shape[1])
    nvj = nbinom.pmf(dataf,n = phi_theta,p= la_pj) #matrix and array?    
    return np.log(nvj).sum()

#PASS DATA ONLY WITH RELEVANT COLUMNS FOR THIS PROBLEM 
#HERE (2854,) 100

def likelihood_P(param, datap,y):
    '''[intercept, gender, 15 cancer types, k genes]'''
    xw = param.p[0]
    #print(datap.head())
    #print('xw: ', xw[0:10],'datap.shape[1]: ',datap.shape[1], 'param.p: ',param.p.shape)
    for i in np.arange(1,datap.shape[1]):
        xw = xw+param.p[i]*datap[datap.columns[i-1]]
    
    #theta = t(param.lm_tht)
    #print('param.lm_tht shape',param.lm_tht.shape)
    theta = np.transpose(param.lm_tht)
    aux = datap.shape[1]
    #print('xw shape(ok)',xw.shape, 'theta shape - wrong ',theta.shape, 'datap shape',datap.shape, 'param p shape',param.p.shape )
    #xw shape (2854,) theta shape (1981, 100) datap shape (2854, 16) param p shape (103,)

    for j in np.arange(0,theta.shape[1]):
        xw = xw + param.p[aux+j]*theta[:,j]
    
    xwy = 0
    for k in np.arange(1,len(y)): 
        xwy = xwy+xw[k]*y[k]
    
    lxp = sum(-np.log(1+np.exp(xw)))
    #page 2 http://www.medicine.mcgill.ca/epidemiology/joseph/courses/EPIB-621/bayeslogit.pdf
    #http://www.utstat.utoronto.ca/reid/sta2201s/2014/feb14-annotated-copy.pdf
    #https://www.statlect.com/fundamentals-of-statistics/logistic-model-maximum-likelihood
    return xwy + lxp

In [8]:
'''
Definition of the posterior distribution and the split of the datasets
Note: the parameters are ok 
'''
def posterior(param,data_F,data_P,y,k):
    j = data.shape[0]
    #print(data_F.head())
    return (likelihood_F(param,data_F)+likelihood_P(param, data_P,y)+priori_l(param)+priori_p(param))

In [9]:
'''
Proposal distribution
'''

#Proposal values for the parameters related to the factor model 
#Repete the parameters related to prediction and only propose
#new values for the factor analysys part
#output is the parameters class 
def proposal_f(current):
    new = parameters(np.random.normal(current.ln,0.005),
                     np.random.normal(current.la_sk,0.005),
                     np.random.normal(current.la_pj,0.005),
                     np.random.normal(current.lm_phi,0.005),
                     np.random.normal(current.lm_tht,0.05), 
                     current.p)
    new.lm_phi[new.lm_phi<0] = 0.00001
    row_sums = new.lm_phi.sum(axis=1)
    new.lm_phi = new.lm_phi / row_sums[:, np.newaxis]
    return new

#Proposal values for the parameters related to logistic regression 
#Repete the parameters related to factor analysis part and propose
#new values for the logistc regression parameters 
#output is the parameters class 
def proposal_p(current):
    new = parameters(current.ln, current.la_sk, current.la_pj, 
                     current.lm_phi, current.lm_tht, 
                     np.random.normal(current.p,0.05))
    return new

In [10]:
'''
Creatint the MCMC for the model
MCMC(
startvalue = initial value for the parameters
iterations = 
data = complete data with all columns 
k = number of latent variables
remove, lr, y = columns names to be removed, presente only in the logistic regression part and y
)
'''
def MCMC(startvalue, iterations, data,k, lr,y):
    '''Splitting dataset'''
    data_P = data[lr]
    data_F = data.drop(lr,axis = 1)
    data_F = data_F.drop(y,axis = 1)
    y = data[y]
    
    '''
    Initialization of the chains
    Note: chain_f has elements from chain_p and vice-versa. Take care to not use incorrectly
    '''     
    chain_f = []
    chain_p = []
    chain_f.append(startvalue)
    chain_p.append(startvalue)
    #chain1[0]=proposalvalues1(startvalue1)
    #chain2[0]=proposalvalues2(startvalue2)
    
    for i in np.arange(1,iterations):
        '''Factor Analysis - Latent Features'''
        #use chain_f or chain_p don't make difference here because
        #the only parameters changed are the logistic regression and 
        #they aren't used in the factor analysis part. 
        param_new_f = proposal_f(chain_f[i-1])
        param_cur_f = chain_f[i-1] 
        print('iteration ',i)
        prob_f = np.exp(posterior(param_new_f,data_F,data_P,y,k)-posterior(param_curr_f,data_F,data_P,y,k))
        if np.random.uniform(0,1,1)<prob_f:
            chain_f.append(param_new_f)
        else:
            chain_f.append(param_cur_f) 
        '''Logistic Regression - Prediction'''
        #chain_f[i] has the most update latent parameters and haven't changed the 
        #prediction parameters from [i-1] iteration
        param_new_p = proposal_p(chain_f[i-1])
        param_cur_p = chain_f[i-1]
    
        prob_p = np.exp(posterior(param_new_p,data_F,data_P,y,k)-posterior(param_curr_p,data_F,data_P,y,k))
        if np.random.uniform(0,1,1)<prob_p:
            chain_p.append(param_new_p)
        else:
            chain_p.append(param_cur_p)     

        return chain_p, chain_f
        
        

In [11]:
'''Non informative prioris: dirichlet has only 1, gamma distribution with 1 average, etc'''
aux = len(lr)+1
data = data.drop(remove,axis = 1)

start = parameters([3,5,2,1], #ln 
                   np.repeat(15,k), #la_sk
                   np.repeat(0.5,data.shape[0]), #la_pj
                   np.repeat(1/k,(data.shape[1]-aux)*k).reshape((data.shape[1]-aux),k) ,#lm_phi
                   np.repeat(1,(data.shape[0])*k).reshape(k,(data.shape[0])), #lm_theta
                   np.concatenate(([-(k+aux)], np.repeat(1,k+aux))))  #p



In [12]:
#data = data.drop(remove,axis=1)
output_p, output_f = MCMC(start,100,data,100,lr,y)




iteration  1
eta 0.13495137094403034
[[1.38324868e-02 1.14498943e-02 5.53623519e-03 ... 1.48403304e-02
  1.00386129e-02 8.75218927e-03]
 [2.41695258e-03 1.07605756e-02 9.23955889e-03 ... 5.05286473e-03
  8.92642215e-06 1.68844441e-02]
 [1.83901666e-02 1.48449644e-02 8.74222374e-03 ... 7.22053588e-03
  1.22592563e-02 8.21512615e-03]
 ...
 [1.09176175e-02 1.48096871e-02 1.04184498e-02 ... 1.28153968e-02
  9.56302026e-03 1.23770148e-02]
 [8.85648484e-03 8.73453254e-03 1.69477650e-02 ... 1.20670412e-02
  1.72470859e-02 9.58122967e-03]
 [2.27989621e-03 7.77862420e-03 1.76586851e-02 ... 1.65085195e-02
  1.01748112e-05 9.50060282e-03]]


C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


ValueError: The input vector 'x' must lie within the normal simplex. but np.sum(x, 0) = [0.12005564 0.11587591 0.13009001 0.10663593 0.12311209 0.08363273
 0.08030722 0.09906182 0.10303526 0.09448247 0.11784922 0.11593324
 0.13134203 0.10948361 0.12002917 0.0999808  0.11731652 0.11438217
 0.10833412 0.10687644 0.10571233 0.12155745 0.11400369 0.11385848
 0.10409386 0.11690772 0.13492519 0.1215503  0.12329899 0.11124145
 0.08339882 0.1239411  0.10006315 0.10642093 0.09211094 0.11126548
 0.11084893 0.10518514 0.10750066 0.11399166 0.11412712 0.12145725
 0.1530334  0.10796945 0.12873784 0.10591827 0.12326188 0.08673312
 0.07875492 0.11907568 0.11399535 0.12442242 0.10534657 0.09663549
 0.07765012 0.1090583  0.10976852 0.10907946 0.12506823 0.07903424
 0.12674772 0.095788   0.11843643 0.07532492 0.12331624 0.10057742
 0.11202195 0.11165977 0.11815626 0.09938879 0.09036491 0.11050428
 0.10307521 0.12236124 0.10675185 0.09690688 0.13176937 0.11937818
 0.09020822 0.07516613 0.10291698 0.13103

In [ ]:
remove

In [ ]:
'''class test:
    def __init__(self, a=[1,2,3], b=0, c=len(a)):
        self.a = a
        self.b = b 
        self.c = c
        
a = test([1,1,1],2,3)
print(a.a,a.b,a.c)

a = test([1,1,1],2,3)
c = test([8,8,8],9,10)
b = [a,c]
print(b[1].b,b[0].b)
b.append(test())
print(b[0].b,b[1].b,b[2].b)
'''

In [20]:
'''a = np.array([1,2,3,4,5,6]).reshape(2,3)
print(a)
b = np.transpose(a)
print(b.shape,b)
print(b[:,1])
phi_theta = np.dot(start.lm_phi, start.lm_tht)
data_F = data.drop(lr,axis = 1)
data_F = data_F.drop(y,axis = 1)
la_pj = np.repeat(start.la_pj, data_F.shape[1]).reshape(start.la_pj.shape[0],data_F.shape[1])

t = nbinom.pmf(data_F,n = phi_theta,p= la_pj)
print(t.shape)
'''
a = np.repeat(0.2,10).reshape(5,2)
b = np.repeat(32,5)
print(a.shape,b.shape)
print(a,b)
phivk = dirichlet.pdf(x = a,alpha = b) #MATRIX #1
phivk

(5, 2) (5,)
[[0.2 0.2]
 [0.2 0.2]
 [0.2 0.2]
 [0.2 0.2]
 [0.2 0.2]] [32 32 32 32 32]


array([35799.48195643, 35799.48195643])

In [39]:
#np.random.gamma(2,9,100).mean()
gamma.pdf(x=12,a = 2,scale = 9)

0.039051427868996555